In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import os

urls = [
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U01.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S01.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S02.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S03.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S04.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U02.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S26.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U03.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S05.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S06.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S07.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S08.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U08.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S27.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S10.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S11.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U09.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U06.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S12.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S13.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S14.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S15.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S16.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S17.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U05.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S18.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U07.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S19.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S20.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S21.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S22.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S29.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S23.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S24.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S28.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S25.htm"
]

# Mapping of state codes to state names
state_code_to_name = {
    "U01": "Andaman and Nicobar Islands",
    "S01": "Andhra Pradesh",
    "S02": "Arunachal Pradesh",
    "S03": "Assam",
    "S04": "Bihar",
    "U02": "Chandigarh",
    "S26": "Chhattisgarh",
    "U03": "Dadra and Nagar Haveli",
    "S05": "Goa",
    "S06": "Gujarat",
    "S07": "Haryana",
    "S08": "Himachal Pradesh",
    "U08": "Jammu and Kashmir",
    "S27": "Jharkhand",
    "S10": "Karnataka",
    "S11": "Kerala",
    "U09": "Lakshadweep",
    "U06": "Delhi",
    "S12": "Madhya Pradesh",
    "S13": "Maharashtra",
    "S14": "Manipur",
    "S15": "Meghalaya",
    "S16": "Mizoram",
    "S17": "Nagaland",
    "U05": "Puducherry",
    "S18": "Punjab",
    "U07": "Daman and Diu",
    "S19": "Rajasthan",
    "S20": "Sikkim",
    "S21": "Tamil Nadu",
    "S22": "Telangana",
    "S29": "Tripura",
    "S23": "Uttar Pradesh",
    "S24": "Uttarakhand",
    "S28": "West Bengal",
    "S25": "Odisha"
}

parent_directory = "state-wise-seat-distribution"
if not os.path.exists(parent_directory):
    os.makedirs(parent_directory)

def extract_state_name(url):
    state_code = url.split("-")[-1].split(".")[0]
    state_name = state_code_to_name.get(state_code, state_code)  # Use mapping to get state name
    return state_name, state_code

def scrape_seat_distribution(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    party_seats = []
    grid_boxes = soup.find_all("div", class_="grid-box")
    for box in grid_boxes:
        party_name = box.find("h4").text
        seats_won = box.find("h2").text
        party_seats.append({"Party": party_name, "Seats": int(seats_won)})
    
    return pd.DataFrame(party_seats)

def save_charts(df, state_name, state_code):
    directory = os.path.join(parent_directory, f"{state_name}-{state_code}")
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Bar chart
    bar_chart_path = os.path.join(directory, f"{state_name}_seat_distribution_bar.png")
    df.plot(kind="bar", x="Party", y="Seats", title=f"Party-wise Seat Distribution ({state_name})")
    plt.ylabel("Seats Won")
    plt.savefig(bar_chart_path)
    plt.close()
    
    # Pie chart
    pie_chart_path = os.path.join(directory, f"{state_name}_seat_distribution_pie.png")
    df.plot(kind="pie", y="Seats", labels=df["Party"], autopct="%1.1f%%", title=f"Party-wise Seat Distribution ({state_name})")
    plt.ylabel("")
    plt.savefig(pie_chart_path)
    plt.close()

for url in urls:
    state_name, state_code = extract_state_name(url)
    df_seat_distribution = scrape_seat_distribution(url)
    save_charts(df_seat_distribution, state_name, state_code)
    print(f"Charts saved for {state_name} ({state_code})")

print("Scraping and chart generation completed.")

Charts saved for Andaman and Nicobar Islands (U01)
Charts saved for Andhra Pradesh (S01)
Charts saved for Arunachal Pradesh (S02)
Charts saved for Assam (S03)
Charts saved for Bihar (S04)
Charts saved for Chandigarh (U02)
Charts saved for Chhattisgarh (S26)
Charts saved for Dadra and Nagar Haveli (U03)
Charts saved for Goa (S05)
Charts saved for Gujarat (S06)
Charts saved for Haryana (S07)
Charts saved for Himachal Pradesh (S08)
Charts saved for Jammu and Kashmir (U08)
Charts saved for Jharkhand (S27)
Charts saved for Karnataka (S10)
Charts saved for Kerala (S11)
Charts saved for Lakshadweep (U09)
Charts saved for Delhi (U06)
Charts saved for Madhya Pradesh (S12)
Charts saved for Maharashtra (S13)
Charts saved for Manipur (S14)
Charts saved for Meghalaya (S15)
Charts saved for Mizoram (S16)
Charts saved for Nagaland (S17)
Charts saved for Puducherry (U05)
Charts saved for Punjab (S18)
Charts saved for Daman and Diu (U07)
Charts saved for Rajasthan (S19)
Charts saved for Sikkim (S20)
Ch